In [1]:
import pandas as pd
import math
import numpy as np
import math
from sklearn.cross_validation import train_test_split # to split the dataset for training and testing

/home/garvit/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# getting the ratings data in pandas dataframe
ratings = pd.read_csv('../Yahoo_movies_multi-criteria/data_movies.txt', sep='\t',names=['user_id', 'criterion1', 'criterion2', 'criterion3', 'criterion4', 'overall', 'movie_id', 'num'])
print(max(ratings.overall)) # implies that ratings are on a scale of 13
# train, test = sample_split(ratings)

13


In [11]:
print("users: ",ratings.user_id.drop_duplicates().count()) #number of users
print("movies: ",ratings.movie_id.drop_duplicates().count()) #number of movies

users:  6078
movies:  976


In [18]:
min(ratings.groupby(["user_id"]).overall.count()) # minimum ratings given by any user


5

In [3]:
# split the datasets into smaller datasets
def split_dataset(ratings):
        df_group = ratings.groupby(['user_id'])
        list_datasets = list() # list containing all the small datasets
        temp = pd.DataFrame()
        count=0
        flag=0
        for key, item in df_group:
            count+=1
            temp = temp.append(item)
            if count==1000:
                if(len(list_datasets)==5 and flag!=1):
                    flag=1
                    count = count - 78
                    continue
                list_datasets.append(temp)
                temp = pd.DataFrame()
                count=0
        return list_datasets

In [4]:
# Spliting a sample into groups userwise
def sample_split(dataFrame):
    df_group = dataFrame.groupby('user_id')
    train = pd.DataFrame()
    test = pd.DataFrame()
    
    for key, item in df_group:
        train, test = split_train_test(item, train, test)
    
    return train, test

In [5]:
# spliting sample in 70% training data and 30% testing data
def split_train_test(dataFrame, train, test):
    temp_train, temp_test = train_test_split(dataFrame, test_size = 0.3, random_state=1212)# in this our main data is split into train and test
    # the attribute test_size=0.3 splits the data into 70% and 30% ratio. train=70% and test=30%
    train = train.append(temp_train)
    test = test.append(temp_test)
    
    return train, test

In [113]:
# list conatining all the datasets
list_datasets = split_dataset(ratings)

In [116]:
# splitting into train and test data
train = list()
test = list()
for i in range(6):
    print(i)
    t1, t2 = sample_split(list_datasets[i])
    train.append(t1)
    test.append(t2)

0
1
2
3
4
5


In [111]:
# function to find similarity between two users based on Manhattan distance
def manhattan_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += abs(row.criterion1_x - row.criterion1_y) + \
                abs(row.criterion2_x - row.criterion2_y) + \
                abs(row.criterion3_x - row.criterion3_y) + \
                abs(row.criterion4_x - row.criterion4_y) + \
                abs(row.overall_x - row.overall_y)
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
manhattan_similarity(train, 1, 3)

0.06666666666666667

In [110]:
# function to find simialrity between two users based on Euclidean distance
def euclidean_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += math.sqrt((row.criterion1_x - row.criterion1_y)**2 + \
                (row.criterion2_x - row.criterion2_y)**2 + \
                (row.criterion3_x - row.criterion3_y)**2 + \
                (row.criterion4_x - row.criterion4_y)**2) + \
                (row.overall_x - row.overall_y)**2
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
euclidean_similarity(train, 1, 3)

0.06319010128182817

In [109]:
# function to find simialrity between two users based on Chebyshev distance
def chebyshev_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += max(abs(row.criterion1_x - row.criterion1_y), \
                abs(row.criterion2_x - row.criterion2_y), \
                abs(row.criterion3_x - row.criterion3_y), \
                abs(row.criterion4_x - row.criterion4_y), \
                abs(row.overall_x - row.overall_y))
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
chebyshev_similarity(train, 1, 3)

0.2222222222222222

In [108]:
# function to predict the rating given by user to item
# neighbours = -1 implies that consider similarity with all possible users
def predict(df, user, item, similarity, neighbours = -1): # df is the train dataset
    neighbours_data_list = list()
    for v in df.user_id.unique():
        if(v==user): # not including the user itself
            continue
        temp = df[df['user_id']==v]
        temp = temp[temp['movie_id']==item]
        if(temp.empty): # user 'v' has not rated the item
            continue
        else:
            rate = temp.iloc[0].overall
        sim = similarity(df, user, v) # find appropriate similarity measure between the two users
        neighbours_data_list.append((sim, rate))
        
    # sort the neighbours_data_list in descending order based on rate
    neighbours_data_list.sort(reverse=True)
    
    # crop the list to the number of neighbours given in the argument
    if(neighbours!=-1):
        length = len(neighbours_data_list)
        neighbours_data_list = neighbours_data_list[:min(neighbours, length)]
    
    # predict the rating using collaborative filtering formula
    numerator = 0
    denominator = 0
    if(len(neighbours_data_list)==0):
        print("We cannot predict")
    for tup in neighbours_data_list:
        numerator += tup[0]*tup[1]
        denominator += tup[0]
    if(denominator==0):
        return 0
    predicted_rating = numerator/denominator
    return predicted_rating
        

In [119]:
# print(test[0].head(26))

In [121]:
# print(train)
print(predict(train[0], 1, 879, euclidean_similarity))
print(predict(train[0], 1, 879, manhattan_similarity))
print(predict(train[0], 1, 879, chebyshev_similarity))

10.37336757999115
9.959976730657361
9.569398389388915


In [126]:
# function to calculate the mean average error between two vectors
def mean_average_error(list1, list2):
    sum1=0
    for a,b in zip(list1,list2):
        if(b==0):            # to handle the unpredicted ratings
            continue
        sum1 += abs(a-b)
    error = sum1/len(list1)
    return error
mean_average_error((1,2),(10,20))

13.5

In [125]:
# function to replace entries in a list with zero/one based on some threshold
def replace_with_zero_one(list1, threshold):
    length = len(list1)
    rlist = list()
    i = 0
    while i < length:
        if(list1[i]==0):   # to handle the unpredicted ratings
            list1[i]=-1
            continue
        if(list1[i]>threshold):
            rlist.append(1)
        else:
            rlist.append(0)
        i += 1
    return rlist

In [127]:
# function to calculate the precision between two lists
# list2 is the predicted list
def precision(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fp = 0 # false positives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==1 and a==0):
            fp+=1
    precision = tp/(tp + fp)
    return precision

# precision([20,5,14], [1,16,20], 13)

In [128]:
# function to calculate the recall between two lists
# list2 is the predicted list
def recall(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fn = 0 # false negatives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==0 and a==1):
            fn += 1
#     print("tp = ",tp)
#     print("fn = ",fn)
    recall = tp/(tp + fn)
    return recall

# recall([20,15,14], [1,16,15], 13)

In [129]:
# function to calculate F1-score between two lists
def f1_score(list1, list2, threshold):
    p = precision(list1, list2, threshold)
    r = recall(list1, list2, threshold)
    f1 = (2*p*r)/(p+r)
    return f1

In [150]:
# predicting on our training dataset
predicted = list() # for storing the predicted ratings
# appending 7 empty lists
predicted.append([])
predicted.append([])
predicted.append([])
predicted.append([])
predicted.append([])
predicted.append([])
predicted.append([])
unpredicted_count = list()
for i in range(6):
    cnt = 0
    for index, row in test[i].iterrows():
        predicted_rating = predict(train[i], row.user_id, row.movie_id,chebyshev_similarity)
#         print(predicted_rating)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
        print(predicted_rating)
    print("predicted %d",i)
    unpredicted_count.append(cnt)
# print(predicted1)

9.925925925925926
11.0
10.117706237424548
10.149111937216027
9.81301482701812
9.569398389388915
10.086199528324284
9.84674418604651
10.720879677175574
11.174829646313299
11.42307235491585
10.243778567800913
11.200840659132757
10.21768592428553
We cannot predict
0
12.6
12.333333333333334
8.373626373626374
11.770310932798395
10.900949796472185
10.03030303030303
10.5
9.882352941176471
11.23906006474044
We cannot predict
0
0
0
10.426298022825913
3.509316770186335
6.770833333333332
0
12.03479990510776
11.639853328749856
10.666666666666666
9.753846153846153
We cannot predict
0
10.82940588460115
9.102564102564102
12.4589552238806
11.087094697961357
11.0
9.842696629213485
11.222378180859442
12.585365853658537
11.142548104292382
0
13.000000000000002
11.0
9.35473046903494
11.594035336159392
8.0
11.08695652173913
11.128672713569852
7.7941787941787934
11.482693290660677
11.639402677331898
10.709116651365848
12.341952264381886
11.163985692785802
10.47808764940239
9.191044776119401
11.0
9.0
11.73824

12.448275862068964
8.875
0
8.526315789473685
12.0
10.504367909788733
0
8.36682615629984
5.301492537313433
5.956521739130435
6.0
We cannot predict
0
11.178670458585147
9.865707434052757
6.6923076923076925
10.386292834890964
0
0
6.572265901327891
8.0
10.85543964232489
11.307692307692307
12.175824175824177
0
10.5
11.598130841121497
10.367346938775512
12.76923076923077
10.0
12.176470588235293
11.5
11.176470588235293
9.923400920756196
11.466666666666667
10.900483759502418
8.08350915839991
9.813707155364312
11.19584796303202
11.715821378555304
11.072563178037834
9.251635412123855
We cannot predict
0
10.40115271752298
10.745839980602982
10.376995636526996
8.0
7.203389830508475
10.551209705861606
12.499999999999998
12.133333333333335
4.9319371727748695
11.703389830508472
6.764705882352941
12.501457725947521
10.714285714285715
12.67307284056994
11.6033198156751
7.463414634146342
10.777993901542512
10.359844810863242
10.941176470588236
12.005621486570892
8.11675528795402
11.659072707663723
13.0


12.504806820243063
12.154296374229714
9.407751312071055
12.386463800133981
0
9.142857142857144
12.524692482993505
11.187999999999999
10.390276189903123
11.434125847483793
13.0
5.0
12.453198655848393
11.999999999999998
8.606349206349206
10.290909090909091
10.167816281855355
10.92436974789916
9.491525423728811
11.23404255319149
11.837672628480867
9.80057636887608
8.762179554436402
9.967741935483872
10.783714348794732
10.970620576874277
10.739009931344834
11.891891891891893
8.956937799043061
6.4423076923076925
7.591836734693878
11.487777395947607
12.200000000000001
8.506335891486707
6.0
10.615384615384615
12.058823529411763
10.0
9.617343134121658
0
10.200000000000001
9.320156971705186
10.664096772881976
6.379057591623035
12.0
10.355140186915886
11.730000000000002
6.6015625
12.509299934573322
10.16923076923077
1.0
13.0
9.0
10.0
11.412371134020619
13.0
10.0
9.0
9.518518518518519
8.0
11.433442362969577
12.829129373474368
0
8.0
11.266333108823245
9.860520659183187
8.07862306325866
10.65606361

8.364471699954331
9.72993411375209
7.02592397359062
11.682312678028035
11.875
3.0
8.452638465784794
11.915602287746905
10.18181818181818
11.005196809672238
10.090909090909092
10.544272059594512
12.552985847428376
12.121281464530892
12.380952380952381
13.0
7.9346882724526235
12.297872088106926
12.58139534883721
10.029858974909796
6.842650321296669
8.671023359288098
9.547480987079346
10.463401321605263
4.338501291989665
7.338396037050598
10.997705477946383
12.142857142857144
4.480519480519481
13.0
8.0
11.040742090019855
11.043478260869565
9.5
4.1886380189366355
We cannot predict
0
13.0
10.499753815854254
9.0
12.054492048663496
11.658188362327529
0
0
11.166901317346566
11.771911653802865
10.801109082302021
11.399651466744116
9.008064516129034
10.801599736125997
10.878787878787879
10.933333333333334
8.7209705372617
10.27912269942529
4.4
11.391134563877735
10.371428571428572
11.329285044049415
12.173208357742624
12.004251589584115
12.275862068965518
0
11.1847288926011
12.106382978723403
12.

12.0
11.307787391841781
0
11.618174151380988
10.88888888888889
10.387627783439882
12.653179190751446
7.2857142857142865
12.244001314780318
9.14942528735632
9.142857142857144
We cannot predict
0
12.0
10.0
6.000000000000001
11.945783132530119
9.912692610505006
0
12.030355594102337
10.880076685456112
11.51036461225245
11.572291437618361
11.789089558246054
0
9.0
10.7199313749097
We cannot predict
0
10.258361994546377
We cannot predict
0
7.741721854304636
11.290909090909093
9.380952380952381
11.377980720446471
9.968697123519458
11.079989015731043
11.363636363636363
12.104597547209522
11.33052039381153
13.0
13.0
6.2028134942098925
7.167775482334815
6.740164067137997
0
11.502972304572895
10.058823529411764
11.925540233584975
9.890563564875492
7.541666666666667
13.0
12.040260928578903
11.88777108930486
10.0
12.486814120305471
11.872340425531915
6.646260716010238
10.858042111541655
10.5997305548066
6.0
6.186666666666667
10.351510309775342
8.296296296296298
12.0
9.03460451977401
7.10132158590308

...

11.5
8.86553179536133
9.790151971603809
10.744376278118613
9.179641502027998
6.999999999999999
0
12.228571428571428
12.188841201716738
11.839285714285717
11.824175518110108
4.0
8.51305069793991
11.756570663926425
6.410958904109592
9.740821142048137
0
10.615384615384615
12.148936170212766
11.397260273972602
11.0
0
0
7.8602442333785625
7.188764044943823
4.666666666666667
11.509054115753381
9.0
10.524590163934427
1.0
9.218631178707223
9.338983050847457
12.10762765235586
9.519022457067372
11.073007470193817
8.0
7.985507246376811
11.0
6.1
8.727353071144497
9.013684791308876
10.508799214209706
10.08817653748041
10.009746588693957
9.542185040643396
10.64753191003433
8.433010788941333
10.511147202126088
3.3665232230301227
9.34082735039873
10.20754716981132
10.0
7.795014626691265
6.999999999999999
12.0
4.923076923076923
11.034198893029862
11.403206171484412
10.794392523364486
12.125288743975595
11.240422721268162
10.563610177628421
11.0
11.5
0
We cannot predict
0
0
5.9655172413793105
10.9545454

0
10.0
0
8.180094786729859
0
13.0
12.954545454545459
11.885245901639342
11.0
We cannot predict
0
8.0
10.78302642113691
6.199999999999998
8.647058823529411
0
8.719867305565796
12.0
5.0
7.731597293418085
8.742331288343559
11.495231831634332
5.897727272727274
5.467532467532467
9.675675675675675
0
7.454545454545454
9.612903225806452
11.201664219285362
11.244181498620442
5.855670103092784
8.402489626556015
11.332160905018949
6.519230769230769
11.198294243070363
8.665512465373961
0
9.159357870007831
9.088888888888887
6.2993630573248405
12.071209800918837
11.767857142857142
8.515406286741053
11.293760831889076
10.43952095808383
11.478080857282027
10.0
10.000000000000002
0
11.130434782608697
7.757196495619526
9.50088339222615
10.752329573320255
7.0
9.352517190908841
12.0
11.571428571428573
9.250000000000002
8.578675282714055
6.252086811352253
10.661532456900243
We cannot predict
0
12.0
1.0
10.14801324503311
0
7.492760858711932
9.399151745868819
10.0
11.650485639286284
9.125
10.619095085706759


10.474137931034484
We cannot predict
0
10.530680824293015
11.967741935483872
0
5.0
11.534972094721889
10.866666666666665
0
8.62533433306848
7.666666666666668
0
0
11.943052116673718
10.846153846153847
9.42666793114894
10.796747967479675
10.224806201550388
10.32469182869488
8.275862068965518
10.383492702566683
0
8.0
We cannot predict
0
0
12.0
12.155963302752292
We cannot predict
0
13.0
9.375
11.320027710883116
11.244394150417826
10.834080717488787
11.352948227247593
0
13.0
0
0
8.941176470588234
8.172413793103448
10.0
0
11.4586043241343
0
9.338709677419356
10.8913950254905
12.0
9.41430568842446
6.803425371905362
12.724725943970768
12.124919177550762
11.998133167392655
11.753623188405799
11.206227723592962
5.727272727272727
10.847999999999997
11.0
9.557184750733139
11.841269841269844
0
11.92454311021415
11.21840182507921
12.266217184301706
9.739810813967068
12.161389486627728
11.902956088543016
11.85805475623557
9.316573908121876
8.802200871758346
10.580628104276025
8.029756140505327
11.99

10.965490802821083
10.282608695652174
12.272435822268346
0
10.698233385410408
11.118251928020566
12.248342507993705
10.964885460045782
2.8
11.45996092174245
7.994764397905759
0
1.0
3.0000000000000004
11.244493392070485
11.651738733399839
11.758620689655169
10.747443762781186
9.473684210526317
10.328191945158526
11.156093781007543
10.11317367275163
9.0
10.698896515770265
11.521558722713333
10.0
10.583973655323822
0
We cannot predict
0
9.666666666666666
0
11.857396280644597
6.964630225080386
7.932432432432432
6.6866438356164375
0
12.000000000000002
0
8.358024691358025
9.943661971830986
0
7.799999999999999
9.0
11.02469135802469
10.096591938659374
7.773195876288659
9.38427393785669
12.444444444444443
0
8.486451511114538
10.5
9.510245901639342
11.374658158614404
10.69558713537771
11.106382978723403
7.7377049180327875
7.683952633728053
8.682801174144851
10.786782881585452
9.448275862068966
8.13972856261567
0
5.757709251101321
8.222222222222223
11.0
11.043412436610705
11.96911433831049
11.474

13.0
10.333333333333334
11.272727272727272
6.0
11.892086330935253
11.068101392154151
11.428571428571429
6.999999999999999
12.8
10.30982019363762
11.934439054920983
11.788280720053265
9.422308998302206
11.480550986663655
12.134267085245112
11.561800573137832
10.2
0
12.022222222222222
0
0
0
10.673968654866302
11.988681948424066
0
12.0
0
10.869565217391305
6.444444444444445
12.529411764705884
8.0
10.235294117647056
7.0
We cannot predict
0
7.0
0
9.038338658146964
9.226860254083485
9.176620076238882
2.311929144755051
11.571428571428571
7.226415094339622
0
0
10.825795853988607
10.767441860465118
10.8338091795097
9.917808219178083
11.100084010052557
11.686643289767296
12.546939476698574
11.919857559851998
11.683508459567857
11.147634452805232
12.49939522225582
10.4375
12.132748642682419
11.247081873578292
8.378378378378379
12.845007451564827
9.393220338983053
5.718309859154929
10.692307692307692
We cannot predict
0
0
0
2.285714285714286
1.0
11.580286483653286
12.713499281953085
8.299065420560

In [100]:
# predicting on our training dataset1
predicted1 = list() # for storing the predicted ratings
for index, row in test1.iterrows():
    predicted_rating = predict(train1, row.user_id, row.movie_id,euclidean_similarity)
    print(predicted_rating)
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted1.append(predicted_rating)
print(predicted1)

10.15841584158416
11.0
10.351666666666667
9.653309807046504
9.990943823404557
9.959976730657361
10.230779198129506
10.1205197831363
10.323383372005729
11.093421327575395
11.614510315128037
10.28037138084387
11.359199641394184
10.24057256384309
We cannot predict
0
Continuing
12.857142857142858
12.6
8.260024092238856
11.505430748359686
11.081177520071366
9.805970149253733
11.125000000000002
8.04
11.190881458966567
We cannot predict
0
Continuing
0
Continuing
0
Continuing
10.289187548773086
2.650812338121027
7.28144662168611
0
Continuing
12.350648862744983
11.859769843653726
11.138461538461538
9.88659793814433
We cannot predict
0
Continuing
10.825941659666457
9.040712468193384
12.5128167611693
11.267767508092339
11.0
10.072436500470367
11.511664686156976
12.572519083969466
11.455735915973262
0
Continuing
13.0
11.0
9.334930198065356
11.887443723003793
8.0
11.045523520485583
11.480644915091446
8.237536656891496
11.732932726962616
12.047204481273564
10.887570718606081
12.461802806476172
11.56

11.077432697263749
10.402061855670102
9.948580074905609
11.25850848461833
10.73772889826917
We cannot predict
0
Continuing
13.000000000000002
10.345488969965468
12.0
0
Continuing
10.0
11.142857142857142
8.77953532266129
6.228826151560178
13.000000000000002
11.681112488025734
5.0
11.272349272349276
10.0
13.0
10.366269899139274
10.364863262416467
12.827828681014276
11.914942128592978
8.637139058386133
13.0
10.350604060939947
12.016100178890879
11.538461538461538
11.060801429869777
11.397296191659978
10.65298631492379
10.258824459613624
8.332171172031712
6.7863901809908835
10.448473284833435
13.0
We cannot predict
0
Continuing
0
Continuing
0
Continuing
0
Continuing
7.0
5.333333333333334
11.4070796460177
7.245508982035928
8.514563106796116
13.0
0
Continuing
2.1241507103150092
11.123373714765847
12.483755480848995
12.42796268361474
10.837102638542078
11.880190816000532
13.0
8.782783969646669
10.0
5.0
11.827668289191026
12.484126984126984
9.1
0
Continuing
8.651094890510947
12.0
10.3631338102

8.397414277684092
12.181250000000002
11.865771812080538
0
Continuing
0
Continuing
10.853658536585366
10.796575739988393
9.456438589035276
13.0
10.121951219512194
8.536263257074332
9.0
8.741016427104723
9.76595744680851
0
Continuing
9.0
11.080000000000002
10.51713930299432
12.326266691442644
11.95894215340207
11.975623367534116
11.128589545367682
0
Continuing
13.000000000000002
12.666666666666666
12.666666666666666
7.105527638190955
0
Continuing
5.52808988764045
12.00361733026149
12.710127324437263
12.625949666890811
10.774441691363425
2.923580214603507
11.839091935541084
12.164556962025317
12.999999999999998
12.000000000000002
11.8
11.734278709164402
11.160565189466924
12.60114663726571
12.381529850746269
12.148936170212766
11.197702558893349
12.131578947368421
12.315758614275188
9.661879231251849
12.166734796354579
10.0
12.0
13.0
We cannot predict
0
Continuing
11.985134684207893
10.791708796764405
0
Continuing
11.5
11.0
11.755102040816325
12.5
11.516384750723283
0
Continuing
12.432631

11.394553072625696
13.0
0
Continuing
0
Continuing
9.0
11.03969292544777
12.65753424657534
3.160919540229885
4.0
11.03247317780473
7.999967928252254
11.985706228972598
11.164086367460186
10.30909090909091
12.373838456040028
5.978511582824864
13.000000000000002
13.0
12.351714075993874
10.3125
6.67309393242526
11.09090909090909
6.793103448275862
9.440751370684406
9.676667723926558
8.209550301344459
10.849050599342316
10.315457413249213
13.0
11.53196846363298
8.42857142857143
6.411764705882352
9.413793103448276
0
Continuing
11.283516627584417
10.537490134175217
13.0
11.766942720954386
11.655370053738727
12.333333333333334
12.491861138913766
8.0
10.654353892160712
9.48253355033274
0
Continuing
11.758620689655173
9.054794520547944
12.527204502814257
11.760626667908749
9.754319907190688
12.417506297229219
10.67413878789464
12.568434886100812
11.919804338371275
11.0
11.846153846153845
12.001460963938044
11.257525083612041
10.22222222222222
8.074074074074074
10.600000000000001
12.0
0
Continuing

KeyboardInterrupt: 

In [158]:
print("unpredicted = ",unpredicted_count[5])
print(len(predicted[5]))
threshold = 9.5
ttest = test[5].overall
list1 = replace_with_zero_one(ttest.tolist(), threshold)
list2 = replace_with_zero_one(predicted[5], threshold)
# mean average error
print("mae = ",mean_average_error(ttest, predicted[5]))
# precision
print("precision = ",precision(list1, list2, threshold))
# recall
print("recall = ",recall(list1, list2, threshold))
# f1 score
print("f1-score = ",f1_score(list1, list2, threshold))

unpredicted =  446
3729
mae =  3.2462202742236244
precision =  0.7648738695859115
recall =  0.6944684528954191
f1-score =  0.7279728199320499
